<a href="https://colab.research.google.com/github/Jakeh33/robotics/blob/main/underactuated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#第一部分 引言



## 第一章 全驱动与欠驱动
大部分机器人均为欠驱动，现在的机器人在模仿自然界生物（人、鸟、昆虫等）动态运动方面还远远不足。

机器人距离像人一样能够灵敏和优秀地执行基本的操作物体任务还有很远的距离。

经典控制技术讲究利用反馈来抵消模型动态，而想要获得出色的动态性能（效率、敏捷、鲁棒性）我们需要设计能够利用模型动态的控制器。

### 欠驱动相关定义
1. 根据牛顿力学，$F=ma$,机械系统的动态为二阶（加速度为位置的二阶导），给出位置向量$\pmb{q}$，速度向量$\pmb{\dot{q}}$，以及时间（很可能）。二阶控制动态系统的一般形式为$\pmb{\ddot{q}}=\pmb{f}(\pmb{q},\pmb{\dot{q}},\pmb{u},t)$。其中$\pmb{u}$是控制向量。称为**欠驱动控制微分方程**。  
当f是满射时，即对任意状态和时间下都存在一个控制量u能够获得期望响应$\pmb{\ddot{q}}$，称为全驱动。（简单讲就是执行器数等于自由度数，通过控制执行器能够达到所有状态）

2. 我们更关心控制仿射（control affine）的情况，一般形式改写为  
$$\tag{1.1.1} \pmb{\ddot{q}}=\pmb{f_1}(\pmb{q},\pmb{\dot{q}},t)+\pmb{f_2}(\pmb{q},\pmb{\dot{q}},t)\pmb{u}$$  
当
$$\tag{1.2.1} rank[\pmb{f_2}(\pmb{q},\pmb{\dot{q}},t)] < dim[\pmb{q}]$$，系统欠驱动。  
注意：输入等其他约束也可能导致系统欠驱动。


#### 例1.1 双摆 
![双摆](http://underactuated.csail.mit.edu/figures/simple_double_pend.svg)
- 利用拉格朗日方程建立标准运动方程
  - $T$：系统总动能(kinetic energy)；$U$：系统总势能(potential energy)
  - $L=T-U$ , $Q_i$对应于$q_i$的广义力。
  - 拉格朗日动态方程：
  $$\tag{1.2.1} \frac{d}{dt}\frac {\partial{L}}{\partial \dot{q}_i}-\frac{\partial{L}}{\partial{q_i}} = Q_i.$$
- 理解拉格朗日力学方程
  - [视频讲解](https://www.bilibili.com/video/BV1gb411y7bq)
  - 将全局方程降为沿x轴方向的一维信息来理解，想要计算系统未来的运动形式，需要知晓系统x轴方向的位置和速度，即$X,\dot{X}$,假设存在一个名为$L$的函数，其值取决于位置和速度以及其他自变量，同时函数值会随着时间而改变。
  - 预测一条从起始状态到终止状态的路径，记录每个路径点的$L$函数值，将其转为关于时间的函数，记函数值的积分为Action，其值会随着路径的变化而改变。如果在图中将路径的改变与action的值表示出来，取action的斜率为0的点作为真实物理系统路径，“principle of stationary action”
  - 为了找到斜率为0的点，需要考虑$L$关于位置的偏导数，得到$\frac{\partial{L}}{\partial{X}}$关于时间的函数，同理得到$\frac{\partial{L}}{\partial{\dot X}}$,计算位置偏导的斜率$\frac{d}{dt}\frac {\partial{L}}{\partial \dot{X}}$
  - $\frac{\partial{L}}{\partial{\dot X}}-\frac{d}{dt}\frac {\partial{L}}{\partial \dot{X}} = 0$ 路径中的小变化不影响action的值，只有斜率为0的点满足，即真实路径。
  - 加入广义力，即$Q_i$改变action的值。
- 双摆运动方程推导
  - 设定$q=[\theta_1,\theta_2]^T$，$\pmb{P_1},\pmb{P_2}$表征$m_1,m_2$小球的位置，系统运动学方程为：
  $$\pmb{P_1} = l_1{s_1\choose-c_1},\pmb{P_2}=\pmb{P_1}+l_2{s_{1+2} \choose -c_{1+2}} $$
  $$\pmb{\dot P_1} = l_1\dot{q_1}{c_1 \choose s_1},\pmb{\dot P_2}=\pmb{\dot P_1}+l_2{(\dot{q_1} +\dot{q_2})}{c_{1+2} \choose s_{1+2}} $$
  - 式中$s_1$表示${sin(q_1)}$;$c_{1+2}$表示$cos(q_1+q_2)$
  - 根据上面的的结果得到动能和势能：
\begin{align*}
      T =& \frac{1}{2} m_1 \dot{\bf p}_1^T \dot{\bf p}_1 + \frac{1}{2} m_2
      \dot{\bf p}_2^T \dot{\bf p}_2 \\
      =& \frac{1}{2}(m_1 + m_2) l_1^2 \dot{q}_1^2 + \frac{1}{2} m_2 l_2^2 (\dot{q}_1 + \dot{q}_2)^2 + m_2 l_1 l_2 \dot{q}_1 (\dot{q}_1 + \dot{q}_2) c_2 \\
      U =& m_1 g y_1 + m_2 g y_2 = -(m_1+m_2) g l_1 c_1 - m_2 g l_2 c_{1+2}
      \end{align*}
  - 计算各项偏导数并代入公式（1.2.1）
  $$\tag{1.2.2} \frac{d}{dt}\frac {\partial{(T-U)}}{\partial \dot{q}_i}-\frac{\partial{(T-U)}}{\partial{q_i}} = Q_i.$$
  \begin{align*}
  (m_1 + m_2) l_1^2 \ddot{q}_1& + m_2 l_2^2 (\ddot{q}_1 + \ddot{q}_2) + m_2 l_1 l_2 (2 \ddot{q}_1 + \ddot{q}_2) c_2 \\
  &- m_2 l_1 l_2 (2 \dot{q}_1 + \dot{q}_2) \dot{q}_2 s_2 + (m_1 + m_2) l_1 g s_1 + m_2 g l_2 s_{1+2} = \tau_1 \\
  m_2 l_2^2 (\ddot{q}_1 + \ddot{q}_2)& + m_2 l_1 l_2 \ddot{q}_1 c_2 + m_2 l_1 l_2 \dot{q}_1^2 s_2 + m_2 g l_2 s_{1+2} = \tau_2
  \end{align*}

- 操纵方程
  - 通过分析简单系统的拉格朗日动态，发现运动方程有着特定的形式，如动能：
  $$\tag{1.2.3} T = \frac{1}{2} \dot{q}^T M(q) \dot{q} $$
  - (1.2.3)中$M$ 是状态相关的惯性矩阵，又名质量矩阵，是正定和对称的。
  - 通用操纵器的运动方程（无运动学循环）
  $$\tag{1.2.4.1} M(q) \ddot{q} + C(q,\dot{q}) \dot{q} = \tau_g(q) +Bu $$
  - (1.2.4.1)中$q$是关节位置矢量，$M$是惯性矩阵，$C$获取科氏力，$\tau_g$是重力矢量。$B$将控制输入$u$映射为广义力。形式上等价于$ma=F$
- 双摆操纵方程
  \begin{align*} 
  \mathbf M(q) =& \begin{bmatrix} (m_1 + m_2)l_1^2 +
  m_2 l_2^2 + 2 m_2 l_1l_2 c_2 & m_2 l_2^2 + m_2 l_1 l_2 c_2 \\ 
  m_2 l_2^2 + m_2 l_1 l_2 c_2 & m_2 l_2^2 \end{bmatrix} \\ 
  \mathbf C(q,\dot q) =&\begin{bmatrix} 0 & -m_2 l_1 l_2 (2\dot{q}_1 + \dot{q}_2)s_2 \\ 
  m_2 l_1 l_2\dot{q}_1 s_2 & 0 \end{bmatrix} \\ 
  \mathbf {\tau_g}(q) =& -g \begin{bmatrix} (m_1 + m_2) l_1 s_1 + m_2 l_2 s_{1+2} \\ m_2 l_2 s_{1+2} \end{bmatrix} , \quad \mathbf B = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix} 
  \end{align*}
  - 三维旋转奇异性的存在使得速度矢量的长度不一定等于位置矢量（使用四元数表示旋转），因此，引入速度矢量$v$，将（1.2.4.1）改写为
  $$\tag{1.2.4.2} \mathbf M(q) \dot{v} + C(q,v) v = \tau_g(q) +\mathbf Bu $$
- 判断
  - $M$可逆，得到加速度的表达式
  $$ \ddot{q} ={\mathbf M}^{-1}(q) \left[ \tau_g(q) + \mathbf Bu - \mathbf C(q,\dot q)\dot q \right]$$
  - $\mathbf B$ 满秩时为全驱动

- 注意点
  - 比较常见的欠驱动为$\mathbf f_2$矩阵不是行满秩，即执行器数小于自由度数
  - 还有些情况执行器数量足够，但是依旧欠驱动，例如人跳跃到空中，虽然有很多的执行器（肌肉）但是依然不能改变运动轨迹。

### 反馈等价
- 当全驱动时，利用反馈将任意控制问题转换为控制线性系统的问题。即当$\mathbf f_2$行满秩，可逆
$$\tag{1.3.1} \mathbf{u} = \pi (q,\dot{q},t) = \mathbf{f_2}^{-1}(q,\dot{q},t) [\mathbf u'-\mathbf {f_1}(q,\dot{q},t)]$$
- $\mathbf u'$是新的控制量，代入公式（1.1.1）得到“feedback equivalence”
$$\tag{1.3.2} \ddot{q} = \mathbf u'$$

#### 例1.2
- 双摆像单摆一样运动（带阻尼），得到动态方程
  \begin{align*} 
  \ddot \theta_1 &= -\frac{g}{l}\sin\theta_1 -b\dot\theta_1 \\
  \ddot\theta_2 &= 0. 
  \end{align*}
- 通过设置控制量实现
$$ \mathbf u = \mathbf B^{-1} \left[\bf C \dot{q} - \tau_g +
{\bf M} \begin{bmatrix} -\frac{g}{l}s_1 - b\dot{q}_1 
\\ 0 \end{bmatrix} \right].$$
- 引入了非线性动力学后，称其为“feedback cancellation”（反馈抵消）或者“dynamic inversion”（动态反演）。全驱动系统可以等价于式（1.3.2）
- 反馈线性化（feedback linearization）：使用反馈将非线性系统转为可控线性系统。能否反馈线性化不等价与是否欠驱动，即使可控线性系统也能是欠驱动的。（在第三章中进行讨论）


---



### 输入和状态约束
- 缺少执行器导致的动态约束，其他动态约束，如：执行器的输出上下限，位置限制等
- 定义：输入和状态约束
  - 动态系统由$ \dot {\bf x} = {\bf f}(\mathbf{x,u},t)$描述
  - 可能受到$\phi(\mathbf{x,u},t) \ge 0$描述的一个或者多个约束
  - 实践中将只取决于输入的约束独立出来，如 $\phi(\bf u) \ge 0$，比状态约束更易处理
  - 空间位置的限制视为一个或多个关于位置的约束，如：$\phi(\bf q) \ge 0$
- 根据欠驱动的广义定义，输入约束会导致系统欠驱动（例如公式（1.3.2）中需要二阶导为2,但是输入约束为$|u|<1$）
- 状态约束通常来说能够减少状态空间维数，减少全驱动的执行器维数需求，但是大多数情况下都不能利用这一点，因为在这种情况下的控制设计很难。
- 两者以相似的方式（不能提供足够的执行器）使控制设计变复杂，通常需要相似的工具来寻找可行控制解。

### 非完整约束 
Nonholonomic constraints
- 约束形式为：$\phi(\mathbf{q,\dot q},t)=0$ 且不能集成到$\phi(\mathbf{q},t)=0$形式的完整约束中
- 不对系统的可能配置进行约束，而是对能够达到这些配置的方式进行约束；完整约束使系统自由度减1，非完整约束不改变系统自由度。
- 与位于轨道上的机器人类比，轨道对应了完整约束：能够写成直接与系统$\bf q$相关的，限制了机器人能够抵达的位置。
- 非完整约束（轮式机器人的无侧滑约束）必然导致系统欠驱动，反之不一定成立。
- 拉格朗日运动方程是形式为$\bf \phi (q,\dot q,\ddot q,\rm {t}) = 0$的约束

#### 例1.4 轮式机器人
- 简单轮式模型，笛卡尔坐标$x,y$表示位置，$\theta$ 表示朝向， $\mathbf q = [x,y,\theta]^T$. 受到一个微分约束避免侧滑
\begin{align*}
\dot x = vcos(\theta) \\
\dot y = v sin(\theta)\\
v = \sqrt {{\dot x^2}+ {\dot y^2}}
\end{align*}
- 或者等价表示为
$$\dot y cos(\theta) - \dot x sin(\theta) = 0 $$
- 这个约束不能集成到位置约束中，因为它对小车能够抵达的$\bf q$做任何限制，只是不能直接侧滑前往。
- 这种影响实现过程的约束即为非完整约束。


### 欠驱动机器人
缺少对于下面列举的欠驱动系统的通用原则：
- 对于一个拥有N个关节和执行的腿式机器人，如果不固定在地面，则系统的自由度包括内部关节和空间6自由度，因为$\bf u \in \mathfrak{R}^N $ 且 $\bf q \in \mathfrak{R}^{N+6}$,满足式（1.1.2）
- （大部分）游泳和飞行机器人
- 操纵机器人时（大多是）欠驱动的。 全驱动机械臂，操纵6自由度物体，可能变成欠驱动。如果能实现并持续力的输出，系统可以认为全驱动。

### 课后练习
1.1 针对波士顿Atlas后空翻的两种状态（1.空翻 2.落地），假设执行器可以产生无限大的扭矩，判断并解释下述说法。
  - 是否可以使用所有关节的角度和角速度表示状态？
    - 我认为不行，因为光凭借角度和角速度不能够表示机器人在空间中的位置
  - 在空翻的过程中，机器人是全驱动的？
    - 我认为不是，在空中无法通过提供扭矩使机器人位置发生改变（除了重力作用）
  - 在站立状态的机器人时全驱动的？
    - 我认为它保持与地面接触时是全驱动（限制了不能离地，自由度减少），一旦能够离开地面（六自由度）就是欠驱动

1.2 假设机器人动态方程为（1.1.1），且在任意时刻的任意状态$\bf x = [q^T,\dot q^T]^T$全驱动。
  - 对任意二阶可导的期望轨迹 ${q_{des}}(t)$，是否能够找到一个控制信号$u(t)$使得$q(t) = q_{des}(t)$，初始位置和速度均为0？
    - 我认为可以，全驱动，轨迹二阶可导，没有约束，可以认为是式（1.3.2）
  - 考虑最简单的全驱动系统：$m \ddot q =u$，已知$u=0$的时候的相图，是否可能实现蓝色和红色轨迹？
    - 相图是根据状态$q,\dot q$绘制的，系统改写成状态方程是
$$\dot x = \left[\begin{matrix}0 & 1\\ 0 & 0 \end{matrix}\right]x+
\left[\begin{matrix}0 \\ \frac 1 m \end{matrix}\right] u $$
    - 蓝线的起点$x(0)=[2,0.5]^T$，当$u=0$时，根据相图，有向x轴正方向的速度，控制量$u$的引入，改变了加速度，但无法做到让其沿蓝线回到原点（可以通过其他轨迹抵达）
    - 红线的起点$x(0)=[-2,0.5]^T$，当$u=0$时，根据相图，有向x轴负方向的速度，通过改变控制量，可以给出一个比较接近红线的轨迹，但是应该不能按照直线抵达（仅猜测）
  - 将公式（1.1.1）扩展成2n维的一阶微分方程$\dot x = f(x,t)$，对任意二阶可导轨迹${x_{des}}(t)$,是否能够找到控制量u使其沿轨迹移动，零初始条件
  $$ f(\bf x,\rm t) = \begin{bmatrix} \bf \dot q \\ {\bf f}_1(\bf q,\dot q,\rm t) + {\bf f}_2(\bf q,\dot q,\rm t)\bf u \end{bmatrix}$$
    - 应该是可以的，初始条件下$\bf x(0) = x_{des}(0)$，即位置和速度均符合轨迹，再结合二阶可导，得到的二阶导数就是需求的控制量？（不确定）

1.3 在例1.1中，每个关节有一个执行器的双摆是一个欠驱动系统，如果直接在$m_2$上施加外力$\mathbf u = [u_1,u_2]^T$，水平和垂直分量，区别就是将公式（1.2.4.2）中的$\bf B$矩阵改为
$$\mathbf B (\mathbf q) = \begin{bmatrix} l_1 c_1 + l_2 c_{1+2} & l_1 s_1 + l_2 s_{1+2} \\ l_2  c_{1+2} & l_2 s_{1+2} \end{bmatrix}$$.
- 此时的系统是否仍为全驱动，如果不是，确定欠驱动的状态。
  - 全驱动时$\bf B$矩阵满秩，确定是否使得存在$det(\mathbf B) = 0$的状态值（$[\theta_1,\theta_2]$）
  - 将等式简化得到${c_1} {s_{1+2}}={s_1}{c_{1+2}}$时不满秩，即$\theta_1 = 0 \quad  or \quad  \theta_2 = 0$

1.4 四旋翼，在纵向平面，$\mathbf q = [x,y,\theta]^T$，重心位置和俯仰角，控制输入$\mathbf u = [u_1,u_2]^T$。
\begin{gather}
m\ddot x = -(u_1+u_2)sin(\theta),\\
m\ddot y = (u_1+u_2)cos(\theta)-mg,\\
I\ddot \theta = r(u_1-u_2)
\end{gather}

- 判断对于什么状态$\bf x = [q^T,\dot q^T]^T$系统欠驱动
  - 当$\theta = 90(deg)$时，明显欠驱动，怎么推导怎么证明再研究

- 对于所有欠驱动的状态，给出一个不能瞬间实现的加速度$\bf \ddot q(q.\dot q)$ 利用运动学等式稍加证明，即将候选加速度代入动态，得到一些个诸如质量为0的矛盾点。


# 第二部分 非线性动态


## 第二章 单摆
### 引入
![单摆](http://underactuated.csail.mit.edu/figures/simple_pend.svg)

- 根据拉氏方程，得到单摆的运动方程
$$m l^2 \ddot\theta(t) + mgl\sin{\theta(t)} = Q.$$
- 广义力Q模拟阻尼扭矩（来自摩擦）和控制输入扭矩
$$Q = -b\dot\theta(t)+u(t)$$

### 李雅普诺夫稳定性
- 根据李雅普诺夫稳定性判断，可以将稳定分为：
  - 局部稳定：
    - 对于不动点周围任意大小$\epsilon$的球1,能够保证存在一个$\delta$范围的球2，其内的每一个点为初始的运动都会保持在球1内。
    - 再简化：从小球内出发的状态不会超出大球
    - 数学表示：$\forall \epsilon > 0 ,\exists \delta >0, if \quad {||x(0)-x^\*||} <\delta \quad then \quad \forall t  \quad {||x(t)-x^\*||} <\epsilon $
  - 局部吸引:
    - 会回到不动点
    - $if \quad x(0)={x^\* + \epsilon}\quad then \quad \lim_{t \to \infty} x(t) = {x^\*} $
  - 局部渐进稳定：
    - 不动点，局部稳定且局部吸引
  - 局部指数稳定
    - $x(0)={x^\* +\epsilon}\quad then \quad \forall t \quad ||x(t)-x^\*||< Ce^{-\alpha t} $
  - 不稳定
- 稳定但不渐进稳定的例子很多，$\dot x =0$ 
- 不稳定但是局部吸引的例子
  \begin{align*} \dot{r} &= r(1-r), \\ \dot\theta &= \sin^2
  \left( \frac{\theta}{2} \right).\end{align*}
  - 两个平衡点：$r^*=0,\theta^{*}=0$和$r^{*} = 1,\theta^*=0$，原点是不稳定点。
  - $r^{*} = 1$ 的不动点吸引其他轨迹，却不稳定。


### 恒定转矩的非线性动力学
考虑不随时间变化的恒定控制量作用下的单摆动力学
$$\tag{2.1.1} m l^2 \ddot \theta+ b \dot \theta +mglsin\theta = u_0 $$
- 上述微分方程由于非线性环节的存在求解极其困难
- 与线性系统形成了鲜明对比，对于一个线性系统
$$\dot{q}=a q\quad \rightarrow \quad q(t) = q(0) e^{at}$$
通过对微分方程的求解可以明显看出$a<0$稳定，$a>0$发散
- 但是可以采用一些图形化处理方法


#### 过阻尼单摆
$b\dot\theta >> ml^2 \ddot \theta$
- 目的是通过设置大的阻尼系数，将系统由二阶降为一阶
$$ ml^2 \ddot\theta + b\dot\theta \approx b\dot\theta = u_0-mgl\sin\theta.$$
- 忽略$\theta$以$2\pi$为周期，将上式改写为
$$\tag{2.1.2} b\dot x = u - mglsinx$$
- 当$u=0$，画出$\dot x$关于 $x$ 的图像  
![曲线](http://underactuated.csail.mit.edu/figures/pend_sinx_annotated.svg)
- 从图中能够看到，系统存在着多个不动点(fixed point),即$\dot x=0$
- 可以在图中看到局部稳定（$x=0$）和不稳定的点($x=\pm \pi$
- 可以确定不动点的吸引域（两个不动点的吸引域不能有重叠）


#### 例2.2 非线性Autapse
- 神经网络中最简单的例子-突触
- 模型数学描述
$\tag{2.2.1} \dot x + x = \tanh(wx)$
- 图像化分析  
![Autapse](http://underactuated.csail.mit.edu/figures/pend_autapse.svg)
  - 对$w<1$，只有一个不动点：原点，
  - 对$w>1$，存在三个不动点，两个稳定一个不稳定
- 是RNN中LSTM模型的基础
  - LSTM 长短期记忆网络，相比于传统RNN，通过遗忘，选择性记忆，对抗在长序列中的梯度消失和梯度爆炸问题。
  - 通过增加门控循环单元（gated recurrent unit），将模型转变为
$$\dot x = -x+\sigma(a_f+b_f u_1+c_f) x + \\
\sigma(a_i x+b_i u_2+ c_i) \tanh(bu_3+c)$$

#### 零扭矩无阻尼单摆
即$b = 0$
1. 假设$u =0 $，系统为二阶，一般可以等价为二维一阶系统
\begin{align*} &\dot x_1 = x_2 & \\ &\dot x_2 = f(x_1,x_2,u),\end{align*}
所以作出的图就不是一条线了，而是一个向量$[\dot x_1,\dot x_2]^T$，此时的向量场就是系统的**相图(phase portrait)**
2. 相图的绘制先从两个坐标轴上开始，确定在$\theta$和$\dot \theta$轴上的每个点对应的向量。
  - 举例说明：在$\theta$轴上，此时向量的x分量为0，y分量为$-\frac g l \sin\theta$，在$\pm \pi$点有不动点。
  - 有稳定不动点，没有渐进稳定点

![相图](http://underactuated.csail.mit.edu/figures/pend_undamped_phase.svg)


#### 轨迹计算
如何在相图中通过少数的向量信息画出轨迹
- 直接通过微分运动方程积分得到结果是比较难的，但是在无控制输入的情况下，我们可以利用物理原理。给出动能和势能的表达式
  $$T=\frac 1 2 I \dot \theta^2,U=-mgl\cos(\theta)$$
  - $I=ml^2$ 转动惯量
  - 总能量$E(\theta,\dot \theta)=T(\dot \theta)+U(\theta)$
  - 对于无阻尼系统，总能量保持不变，即
\begin{gather*}  
E(\theta(t),\dot\theta(t)) = E(\theta(0),\dot\theta(0))= E_0 
\\ \frac{1}{2} I \dot\theta^2(t) - mgl\cos(\theta(t)) = E_0  
\\ \dot\theta(t) = \pm \sqrt{\frac{2}{I}\left[E_0 + mgl\cos\left(\theta(t)\right)\right]} 
\end{gather*}
  - 系统对于$\theta_{max}$有限制
$$ \theta_{max} =\begin{cases} \cos^{-1}\left( -\frac{E_0}{mgl} \right), & E_0 < mgl \\ \pi, & \text{otherwise}. \end{cases}$$
  - 当然这只是理论上的，实际不可能出现超出限制的总能量
- 对于最大总能量$E=mgl$，这条轨道到达了不稳定的平衡点，称为同宿轨道（homoclinic orbit）

#### 有阻尼单摆
- 不动点渐进稳定
  - 是否指数稳定？将系统在不动点附近线性化，指数稳定的时不变非线性系统必须有稳定的线性化结果。
- 如果施加一个时变的输入$u=\pi(\theta ,\dot \theta)=2mgl\sin(\theta)$使得垂直位置稳定。
- 如何施加一个最小的外力，使得垂直不动点稳定？——控制设计应该考虑的方式，方案一使用动态规划。

### 能量成型控制（Energy-shaping control）
将控制器和被控对象都视为能量转换设备
- 首先分析总能量为
$$E = \frac{1}{2} m l^2 \dot\theta^2 - mgl\cos\theta.$$
- 求对时间的导数
\begin{align*} \dot{E} =&
      ml^2\dot\theta \ddot\theta + \dot\theta mgl\sin\theta \\ =& \dot\theta
      \left[ u - mgl\sin\theta \right] + \dot\theta mgl\sin\theta \\ =&
      u\dot\theta. \end{align*}
- 对系统能量进行控制
  - 与$\dot \theta$同向施加外力，能量增加
  - 与$\dot \theta$反向施加外力，能量减小（阻尼）
- 期望总能量$E^d = mgl.$
- 得到能量误差 $\tilde{E} = E - E^d$且$\dot{\tilde{E}} =\dot{E} = u\dot\theta.$
- 设计反馈控制器$u = -k \dot\theta \tilde{E},\quad k>0.$
- 得到$\dot{\tilde{E}} = - k \dot\theta^2 \tilde{E}.$
- 对于固定$\dot \theta \neq 0$，误差趋于0
- 此时的控制能使得不稳定平衡点局部吸引但依旧不稳定。
- 但是这个控制器够简单，且对模型参数$m,g.l$的误差鲁棒。


### 练习
1. 对下面的一阶系统，根据图判断平衡点的稳定性。
$$ \dot x = \begin{cases} - x^5 + 2 x^3 - x & \text{if} \quad x \leq 1, \\ 0 & \text{if} \quad 1 < x \leq 2, \\ - x + 2 & \text{if} \quad x > 2, \end{cases}$$
  - $x^* = -1$ 两边的导数都大于0，不稳定
  - $x^* = 0$ 指数稳定
  - $x^* = 1$ 不稳定（左端不稳定）/稳定？不会小于0
  - $x^* \in (1,2]$ 稳定
2. 对一阶系统$\dot x = f(x) = - x^3 + 4x^2 +11x - 30.$
  - 画图判断不动点，并确定稳定性
    - $x^* = -3$ 渐进稳定
    - $x^* = 2$ 不稳定
    - $x^* = 5$ 渐进稳定
  - 增加一个额外项$w$，动态改为$\dot x = f(x) + w$，做出分叉图，确定w作用下的稳定和不稳定不动点个数
    - 稳定点从$2 \to 1$ 不稳定点从$1 \to 0$
3. 动态方程未知，只知$f(x)$连续且正好有三个稳定不动点。
  - 判断不稳定不动点的最小和最大值。
    - 最小2个，$x \to -\infty, f(x) \to \infty$；$x \to \infty, f(x) \to -\infty$
    - 最多4个，$x \to -\infty, f(x) \to -\infty$；$x \to \infty, f(x) \to \infty$
4. 对于系统
\begin{align*} \dot x_1 &= x_1 (1-|\mathbf x|) + x_2 \frac{x_1-|\mathbf x|}{2|\mathbf x|}, \\ \dot x_2 &= x_2 (1-|\mathbf x|) - x_1 \frac{x_1-|\mathbf x|}{2|\mathbf x|}, \end{align*}
- 找出不动点
  - $[x_1,x_2] = [1,0]$
- 判断是否吸引和稳定
  - 吸引是肯定的
  - 稳定
- 找到和其很相关的在本章中介绍的系统。
  - 应该是那个吸引但不稳定的系统
5. 针对单摆系统(根节点按照水平谐波左右振荡)$m l^2 \ddot \theta + m g l \sin \theta = m l \omega^2 h \sin (\omega t) \cos \theta + u.$，设计控制律，使得单摆以恒定角速度旋转。
- 给出期望闭环动态$\ddot \theta =f(\dot \theta)$
  - 。。。

# 第三部分 非线性规划和控制

## 第七章 动态规划（DP）
- 以无阻尼单摆的相图为例，假设$u=2mgl\sin\theta$，作用等价于重力反向，则稳定点从$\theta = 0$ 移动到$\theta = \pi$，表现在相图中就是右移。

- 但是实际上对于二阶系统，我们通过施加u并不能改变向量场的位置，起点坐标$[\theta,\dot \theta]$,终点坐标$[\dot \theta,\ddot \theta]$，通过改变控制输入$u$,只能改变$\ddot \theta$，即改变终点的y坐标值。

### 将控制设计视作优化问题
定义轨迹$x(·),u(·)$，设计惩罚(代价)/奖励函数，增加约束信息。



#### 例7.1 双积分系统
- 系统模型
$$\ddot q =u,|u| \leq 1$$
- 目标： 使到达目标状态（$$）的时间最小
- 举例理解：

# Drake
[官方文档](https://drake.mit.edu/)
1. 定位：C++工具包，拥有Python接口，分析机器人动态，建立控制系统。
2. 尚未确定安装方式，根据示例安装在notebook中